In [1]:
import pandas as pd
import duckdb
import glob
import os

In [2]:
imd = pd.read_excel('data/File_1_-_IMD2019_Index_of_Multiple_Deprivation.xlsx', sheet_name='IMD2019')
imd.columns = imd.columns.str.strip().str.lower().str.replace(" ", "_")
imd = imd.rename(columns={"lsoa_code_(2011)": "lsoa11_code", 
                          "index_of_multiple_deprivation_(imd)_decile": "imd_decile"})[["lsoa11_code","imd_decile"]]
imd.to_parquet("data/processed/imd_2019.parquet", index=False)

In [47]:
file_path = r'C:\Users\khize\OneDrive\Documents\DataAnalytics\dataProjectsPortfolio\respiratoryAdmissions\data\raw\artificial_hes_apc_202302_v1_full\artificial_hes_apc_202302_v1_full'
con = duckdb.connect("data/pipeline.duckdb")

files = [os.path.join(file_path, f) for f in os.listdir(file_path) if f.endswith('.csv')]
files = files[-3:]

con.execute(f"""
COPY (
    SELECT
        FYEAR AS financial_year,
        --PARTYEAR,
        EPIKEY AS episode_key,
        EPIDUR AS episode_duration,
        EPIEND AS episode_end_date,
        EPIORDER AS episode_order,
        EPISTART AS episode_start_date,
        PSEUDO_HESID AS pseudo_hes_id,
        ADMIDATE AS admission_date_spell,
        ADMIMETH AS admission_method,
        ADMISORC AS admission_source,
        CCG_GP_PRACTICE as ccg_gp_practice,
        DIAG_4_01 AS primary_diagnosis,
        DIAG_4_02 AS secondary_diagnosis,
        DIAG_4_03 AS tertiary_diagnosis,
        ETHNOS AS ethnicity,
        FAE_EMERGENCY AS finished_admission_episode_emergency,
        RESCTY_ONS AS county_of_residence,
        SEX AS sex,
        DISDATE AS discharge_date_spell,
        SPELBGIN AS spell_begin_indicator,
        SPELDUR AS spell_duration,
        SPELDUR_CALC AS spell_duration_calc,
        SPELEND AS spell_end_flag,
        STARTAGE AS age_episode_start,
        LSOA11 AS lsoa11_code
    FROM read_csv_auto({files}, SAMPLE_SIZE=-1)
) TO 'data/processed/data_3yrs.parquet' (FORMAT PARQUET);
""")

con.execute("""
COPY (
  SELECT A.*, B.imd_decile
  FROM 'data/processed/data_3yrs.parquet' A
  LEFT JOIN 'data/processed/imd_2019.parquet' B
  ON A.lsoa11_code = B.lsoa11_code
) TO 'data/processed/data_3yrs_imd.parquet' (FORMAT PARQUET);
""")

In [48]:
df = pd.read_parquet(r"data\processed\data_3yrs_imd.parquet", engine="pyarrow")

## **EDA**

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3000000 entries, 0 to 2999999
Data columns (total 26 columns):
 #   Column                                Dtype  
---  ------                                -----  
 0   financial_year                        int64  
 1   episode_key                           int64  
 2   episode_duration                      float64
 3   episode_end_date                      object 
 4   episode_order                         int64  
 5   episode_start_date                    object 
 6   pseudo_hes_id                         object 
 7   admission_date_spell                  object 
 8   admission_method                      object 
 9   admission_source                      int64  
 10  ccg_gp_practice                       object 
 11  primary_diagnosis                     object 
 12  secondary_diagnosis                   object 
 13  tertiary_diagnosis                    object 
 14  ethnicity                             object 
 15  finished_admiss

Multiple episodes per spell

In [ ]:
spell_counts = (
    df.groupby("pseudo_hes_id")
      .agg(n_rows=("episode_key","count"),
           n_unique_admissions=("admission_date_spell","nunique"))
)
spell_counts[spell_counts["n_rows"] != spell_counts["n_unique_admissions"]]

,n_rows,n_unique_admissions
pseudo_hes_id,,
TEST0038wT3jj50caBQnJ0OkVKJ2VuXn,2,1
TEST005APsRAinbMVQEqYRZlkQGTBPSf,5,4
TEST007d3Gvg3GNnGksjTvQjLup9iO0D,3,2
TEST009N40sWT6c6ALvoTi9HlkVC8JIn,18,17
TEST00EOoFKQiKHqDYU6UO8YpBh15KjK,46,39
...,...,...
TESTzzqbKQYYnO5CoIrfB5b1k1Zrrg10,5,4
TESTzzrM5SZZuplRhGrV3ZDcof3k3cXa,3,2
TESTzzsfRXBOJYFvQ7Guy2mSXEbLoKfW,30,21


In [62]:
pid = 'TEST000GPDDuWdpt8xc2yxAfCBgqHqZJ'  #"TEST009N40sWT6c6ALvoTi9HlkVC8JIn" #"TEST007d3Gvg3GNnGksjTvQjLup9iO0D"
cols = ["episode_key","episode_order","admission_date_spell","admission_method","episode_start_date","episode_end_date","discharge_date_spell",
        "spell_begin_indicator","spell_duration","spell_duration_calc","spell_end_flag"]                         

df.loc[df["pseudo_hes_id"]==pid, cols] \
  .sort_values(["admission_date_spell"])

,episode_key,episode_order,admission_date_spell,admission_method,episode_start_date,episode_end_date,discharge_date_spell,spell_begin_indicator,spell_duration,spell_duration_calc,spell_end_flag
2247205,615611084130,1,2021-04-01,11,2021-04-01,2021-04-01,2021-04-01,2.0,0.0,24.0,Y
2247202,644841684357,1,2021-04-13,21,2021-05-19,2021-06-01,2021-06-01,0.0,0.0,3.0,N
2247203,692601483072,1,2021-04-20,21,2021-04-20,2021-04-20,2021-04-20,2.0,0.0,0.0,Y
2247204,175456608888,1,2021-04-30,11,2021-04-30,2021-04-30,2021-05-28,0.0,0.0,0.0,Y
2294206,67605149287,1,2021-05-06,11,2021-05-17,2021-05-30,2021-05-30,2.0,0.0,0.0,Y
2247201,623012176094,1,2021-05-22,21,2021-05-28,2021-06-17,None,2.0,2.0,22.0,Y


In [ ]:
df_test = df[df['pseudo_hes_id'] == 'TEST000GPDDuWdpt8xc2yxAfCBgqHqZJ'] # TEST007d3Gvg3GNnGksjTvQjLup9iO0D/TEST009N40sWT6c6ALvoTi9HlkVC8JIn
df_test.sort_values('admission_date_spell')

,financial_year,episode_key,pseudo_hes_id,admission_date_spell,admission_method,admission_source,ccg_gp_practice,primary_diagnosis,secondary_diagnosis,tertiary_diagnosis,...,county_of_residence,sex,discharge_date_spell,spell_begin_indicator,spell_duration,spell_duration_calc,spell_end_flag,age_episode_start,lsoa11_code,imd_decile
2243945,2122,615611084130,TEST000GPDDuWdpt8xc2yxAfCBgqHqZJ,2021-04-01,11,98,W2U3Z,J039,H522,None,...,E99999999,1,2021-04-01,2.0,0.0,24.0,Y,64.0,E01017427,7.0
2243942,2122,644841684357,TEST000GPDDuWdpt8xc2yxAfCBgqHqZJ,2021-04-13,21,19,14Y,R104,None,E877,...,E10000011,1,2021-06-01,0.0,0.0,3.0,N,64.0,E01010003,1.0
2243943,2122,692601483072,TEST000GPDDuWdpt8xc2yxAfCBgqHqZJ,2021-04-20,21,19,15C,R074,Z532,None,...,E99999999,1,2021-04-20,2.0,0.0,0.0,Y,64.0,E01030277,1.0
2243944,2122,175456608888,TEST000GPDDuWdpt8xc2yxAfCBgqHqZJ,2021-04-30,11,51,02Y,M809,E86X,None,...,E99999999,1,2021-05-28,0.0,0.0,0.0,Y,64.0,E01025868,8.0
2247671,2122,67605149287,TEST000GPDDuWdpt8xc2yxAfCBgqHqZJ,2021-05-06,11,19,18C,N390,Z907,J449,...,E99999999,1,2021-05-30,2.0,0.0,0.0,Y,64.0,None,NaN
2243941,2122,623012176094,TEST000GPDDuWdpt8xc2yxAfCBgqHqZJ,2021-05-22,21,19,11N,M545,None,Z507,...,E99999999,1,None,2.0,2.0,22.0,Y,64.0,E01016475,4.0
